In [ ]:
# Install canvasapi module

!pip install canvasapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 653.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
  Created wheel for canvasapi: filename=canvasapi-3.2.0-py3-none-any.whl size=113293 sha256=f7ed68491352b853464c5568283cdee1f79f9f8e7d1dc9d44587ddaaa9e841d3
  Stored in directory: /root/.cache/pip/wheels/a7/34/d2/a22bfe4552157389421694718e36b676045006d5141fa49c4d
Successfully built canvasapi


In [ ]:
# Run this cell to generate report
def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'

def get_assigment_submissions(canvas, course_id, assignment_id):
  course = canvas.get_course(course_id)
  assignment = course.get_assignment(assignment_id)
  submissions = [x for x in assignment.get_submissions(include=["submission_comments"])]
  return submissions

def get_user_by_id(canvas, user_id):
  user = canvas.get_user(user_id)
  return(user)

def get_student_users_by_course(canvas, course_id):
  course = canvas.get_course(course_id)

  student_dict = {}
  enrollments = [x for x in course.get_enrollments() if x.type == "StudentEnrollment"]
  for e in enrollments:
    student_dict[e.user_id] = {
        "login_id": e.user["login_id"],
        "sis_user_id": e.user["sis_user_id"],
        "sortable_name": e.user["sortable_name"]
    }
  return student_dict

def generate_report(canvas, course_id, assignment_id, student_dict, submissions, user_group_map):

  rows = []
  for i, sub in enumerate(submissions):
    try:

      row = {
          "login_id": student_dict[sub.user_id]["login_id"],
          "sis_user_id": student_dict[sub.user_id]["sis_user_id"],
          "sortable_name": student_dict[sub.user_id]["sortable_name"],
          "workflow_state": sub.workflow_state,
          "score": sub.score,
          "seconds_late": sub.seconds_late,
          "academic_advisor": user_group_map[sub.user_id],
          "url": "https://liverpool.instructure.com/courses/{0}/gradebook/speed_grader?assignment_id={1}&student_id={2}".format(course_id, assignment_id, sub.user_id)
      }
      #row["url"] = "https://liverpool.instructure.com/courses/{0}/gradebook/speed_grader?assignment_id={1}&student_id={2}".format(course_id, assignment_id, sub.user_id)
      rows.append(row)

    except:
      continue
  df = pd.DataFrame(rows)
  return df

def create_user_group_map(course):
  user_group_dict = {}
  groups = [x for x in course.get_groups()]
  for group in groups:
    for member in [x for x in group.get_memberships()]:
      user_group_dict[member.user_id] = group.name
  return user_group_dict

# important
from canvasapi import Canvas
import pandas as pd

API_URL = "https://canvas.liverpool.ac.uk"
API_KEY = "15502~SwOLyK0SBOhLZktbCda0mmxmtZxTern1znfSvsMXfkOFqZLDfBPOVgYZsKD0NRQr"

# create a canvas object
canvas = Canvas(API_URL, API_KEY)

import datetime
datestamp = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
fname = "{0}_assessment_report.xlsx".format(datestamp) # Generate dynamic filename

# Get course and assignments
#course_id = 45768 #LIFE223 202122
#course_id = 58609 #LIFE223 202223
course_id = 69023 #LIFE223 202324
assignments = [x for x in canvas.get_course(course_id).get_assignments()]
assignments_list = [{"id": a.id, "label": a.name[:30]} for a in assignments if "LIFE223." in a.name and a.workflow_state != "unpublished" ]

# create an input dictionary using course and assignment ids
# {course_id: {"assignments": [{"id": assignment_id, "label": <label>}]}
assignments_dict = {
    course_id: {"assignments": assignments_list},
}

with pd.ExcelWriter(fname) as writer:
  for course_id in assignments_dict:
    print("Getting user data for course {0}. This takes a about 30 sec ...".format(course_id))
    student_dict = get_student_users_by_course(canvas, course_id)
    course = canvas.get_course(course_id)
    user_group_map = create_user_group_map(course)
    assignments = assignments_dict[course_id]["assignments"]
    for assignment in assignments:
      print(course_id, assignment)
      submissions = get_assigment_submissions(canvas, course_id, assignment["id"])
      df = generate_report(canvas, course_id, assignment["id"], student_dict, submissions, user_group_map)
      df = df.sort_values(by=["sortable_name"])


      df.loc[df["workflow_state"]=="submitted", "workflow_state"] = "ungraded"
      df.loc[(df["workflow_state"]=="graded") & (df["score"].isnull()), "workflow_state"] = "EX"


      df.to_excel(writer, sheet_name=assignment["label"], index=False)
      df.style.format({'url': make_clickable})

      summary = pd.DataFrame(df.groupby(by=["academic_advisor"])["score"].mean())
      summary.to_excel(writer, sheet_name="{} - SuperSummary".format(assignment["label"][:11]), index=True)



Getting user data for course 69023. This takes a about 30 sec ...
69023 {'id': 256081, 'label': 'LIFE223.1.1 (50%) - Essay (12.'}
69023 {'id': 254667, 'label': 'LIFE223.1.3 (50%) - Job Applic'}
69023 {'id': 256080, 'label': 'LIFE223.1.3 (50%) - Lay Summar'}
69023 {'id': 254665, 'label': 'LIFE223.1.4 (50%) - Scientific'}
69023 {'id': 254666, 'label': 'LIFE223.1.5 (50%) - Presentati'}


In [ ]:
xls = pd.ExcelFile(fname)
xls.sheet_names

['LIFE223.1.1 (50%) - Essay (12.',
 'LIFE223.1.1 - SuperSummary',
 'LIFE223.1.3 (50%) - Job Applic',
 'LIFE223.1.3 - SuperSummary',
 'LIFE223.1.3 (50%) - Lay Summar',
 'LIFE223.1.4 (50%) - Scientific',
 'LIFE223.1.4 - SuperSummary',
 'LIFE223.1.5 (50%) - Presentati',
 'LIFE223.1.5 - SuperSummary']

In [ ]:
with pd.ExcelWriter('LIFE223_202122_MODERATION_RECORD.xlsx') as writer:
  for sheet in xls.sheet_names:
    if "%" in sheet:
      print(sheet)
      df = pd.read_excel(xls, sheet)
      df = df[df["workflow_state"] == "graded"]
      new = df.sample(frac=0.1)
      half = int(len(new)/2)
      new["Moderator"] = "Wattret, Gemma"
      new.iloc[half:]["Moderator"] = "Treharne, Robert"
      new["score appropriate?"] = ""
      new["moderator comments"] = ""
      new.to_excel(writer, sheet_name=sheet, index=False)

LIFE223.1.1 (50%) - Please do 
LIFE223.1.1 (50%) - Essay (12.


<ipython-input-5-4686b598b93b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.iloc[half:]["Moderator"] = "Treharne, Robert"
<ipython-input-5-4686b598b93b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.iloc[half:]["Moderator"] = "Treharne, Robert"


LIFE223.1.2 (50%) - Job Applic
LIFE223.1.3 (50%) - Lay Summar


<ipython-input-5-4686b598b93b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.iloc[half:]["Moderator"] = "Treharne, Robert"
<ipython-input-5-4686b598b93b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.iloc[half:]["Moderator"] = "Treharne, Robert"


LIFE223.1.4 (50%) - Scientific
LIFE223.1.5 (50%) - Presentati


<ipython-input-5-4686b598b93b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new.iloc[half:]["Moderator"] = "Treharne, Robert"
